In [1]:
from collections import Counter
import omero.clients
from tqdm import tqdm

client = omero.client('localhost')
session = client.createSession(USERNAME, PASSWORD)
client.enableKeepAlive(60)
qs = client.getSession().getQueryService()

def batchgen(arr, size):
    for n in range(0, len(arr), size):
        yield arr[n:n + size]

In [2]:
project_ids = omero.rtypes.unwrap(qs.projection("SELECT id FROM Project WHERE name LIKE 'idr0071%'", None))

pixel_ids = []
for p in project_ids:
    h = client.submit(omero.cmd.FindChildren({'Project': p}, ['Pixels']),loops=120,ms=1000)
    r = h.getResponse()
    pixel_ids.append(r.children['ome.model.core.Pixels'])
    h.close(True)

In [3]:
pixel_sizes = []
for pxids in pixel_ids:
    for batch in tqdm(list(batchgen(pxids, 1000))):
        params = omero.sys.ParametersI()
        params.addIds(batch)
        pixel_sizes.extend(omero.rtypes.unwrap(qs.projection('SELECT sizeX, sizeY, sizeZ, sizeC, sizeT FROM Pixels WHERE id in (:ids)', params)))

c = Counter(tuple(p) for p in pixel_sizes)

for k in c:
    print(f'{k}:\t{c[k]}')

100%|██████████| 45/45 [00:03<00:00, 13.98it/s]

(1024, 1024, 1, 1, 5):	83140
(1024, 1024, 1, 2, 1):	19186
(1024, 1024, 1, 1, 1):	26268
(1024, 1024, 9, 5, 1):	12790
(1024, 1024, 1, 5, 1):	36095
(2048, 2048, 1, 1, 5):	12791
(2048, 2048, 6, 5, 1):	4264
(2048, 2048, 1, 1, 1):	4264
(2048, 2048, 1, 5, 1):	26
(1024, 1024, 1, 1, 2):	6396
(1024, 1024, 12, 5, 1):	686
